In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from nltk.stem.snowball import SnowballStemmer
import nltk
from nltk.stem import WordNetLemmatizer 
import string
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [32]:
df = pd.read_csv('../data/mostly_cleaned.csv')

In [71]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48890 entries, 0 to 48889
Data columns (total 33 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            48890 non-null  int64  
 1   age                   48890 non-null  int64  
 2   status                48890 non-null  object 
 3   sex                   48890 non-null  object 
 4   orientation           48890 non-null  object 
 5   body_type             48890 non-null  object 
 6   diet                  48890 non-null  object 
 7   drinks                48890 non-null  object 
 8   drugs                 48890 non-null  object 
 9   education             48890 non-null  object 
 10  ethnicity             44945 non-null  object 
 11  height                48890 non-null  float64
 12  job                   48890 non-null  object 
 13  offspring             48890 non-null  object 
 14  pets                  48890 non-null  object 
 15  religion           

In [33]:
essays = df.iloc[:, 19:29]
essays.fillna(".", inplace=True)
essays['essays_combined'] = essays['essay0']
for col in essays.columns[1:]:
    essays['essays_combined'] += essays[col]

In [34]:
trimmed_df = df.drop(columns={'Unnamed: 0', 
                 'sign', 
                 'religion', 
                 'essay0', 
                 'essay1', 
                 'essay2', 
                 'essay3', 
                 'essay4', 
                 'essay5', 
                 'essay6', 
                 'essay7', 
                 'essay8', 
                 'essay9',
                 'smokes'})

In [35]:
dummy_df = pd.get_dummies(trimmed_df)


In [ ]:
dummy_df.

In [118]:
for col in list(dummy_df.columns):
    if dummy_df[col].dtype != 'uint8':
        print(col, dummy_df[col].dtype)

age int64
height float64
essay_vector object


In [36]:
tfidf = TfidfVectorizer(max_features=1000, 
                        stop_words='english', 
                        strip_accents='unicode', 
                        analyzer='word')
Z = tfidf.fit_transform(essays.essays_combined)

In [52]:
vector_df = pd.DataFrame(Z.toarray())

In [113]:
vector_df.iloc[0:10,0:1000].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 1000 entries, 0 to 999
dtypes: float64(1000)
memory usage: 78.2 KB


In [66]:
vector_df[0:1]

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.173365,0.0,0.0,...,0.0,0.0,0.0,0.064153,0.0,0.0,0.0,0.0,0.0,0.0


In [119]:
combined_df = dummy_df.drop(columns='essay_vector').join(vector_df)

In [120]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48890 entries, 0 to 48889
Columns: 8065 entries, age to 999
dtypes: float64(1001), int64(1), uint8(7063)
memory usage: 703.1 MB


In [121]:
y=df['smokes']
X=combined_df

In [122]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((36667, 8065), (12223, 8065), (36667,), (12223,))

In [123]:
GB = GradientBoostingClassifier()
GB.fit(X_train, y_train)
GB_pred = GB.predict(X_test)

GB.score(X_test, y_test)

0.7422891270555511

In [124]:
RF = RandomForestClassifier()
RF.fit(X_train, y_train)
RF_pred = RF.predict(X_test)

RF.score(X_test, y_test)

0.7336987646240694

In [ ]:
gb_gridsearch = GridSearchCV(estimator=GradientBoostingClassifier(),
                                param_grid = {'max_depth': [2, 4, 8],
                                             'min_samples_split': [2, 4, 8],
                                             'subsample': [1, 0.5, 0.25, 0.1]},
                                cv=5, verbose=0, return_train_score=True)
gb_gridsearch.fit(X_train, y_train)
df_gridsearch = pd.DataFrame(gb_gridsearch.cv_results_)